In [13]:
import os
import sys
home_dir = "../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd 

In [14]:
task = "popu_freq"
result_df = pd.read_csv(home_dir+f"models/dbnsfp/postprocessed_outputs/{task}.tsv", sep="\t")
print(result_df.columns)
print(result_df["class"].value_counts())
print(result_df.shape)

Index(['snp_id', 'gene_symbol', 'chrom_acc_version', 'chrom_pos', 'ref_allele',
       'alt_allele', 'prot_acc_version', 'prot_pos', 'wt', 'mut',
       'wt_population', 'mut_poulation', 'wt_freq', 'mt_freq', 'class',
       'chrom', 'SIFT_score', 'Polyphen2_HVAR_score', 'MetaRNN_score',
       'REVEL_score', 'MVP_score', 'CADD_raw_score',
       'integrated_fitCons_score', 'phyloP17way_primate_score',
       'phastCons17way_primate_score', 'bStatistic_score'],
      dtype='object')
Singleton     422813
Ultra-rare    286752
Rare           25554
Common         14429
Name: class, dtype: int64
(749548, 26)


In [3]:
# computing number of supervised models score available for each row
supervised_method_cols = ['MetaRNN_score', 'MVP_score', 'SIFT_score', 'Polyphen2_HVAR_score', 'CADD_raw_score', 'REVEL_score']
def get_num_of_methods_have_prediction_on_this_row(row):
    n = 0
    for col in supervised_method_cols:
        # print(pd.isna(row[col]))
        if not pd.isna(row[col]):
            n += 1
    # print(n)
    return n

for i in range(result_df.shape[0]):
    if i%10000==0: print(i)
    result_df.loc[i, "n_methods_having_preds"] = get_num_of_methods_have_prediction_on_this_row(result_df.loc[i])

print(result_df.shape)
print(result_df.columns)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
(749548, 26)
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq', 'class', 'chrom', 'SIFT_score',
       'Polyphen2_HVAR_score', 'MetaRNN_score', 'REVEL_score', 'MVP_score',
       'CADD_raw_score', 'integrated_fitCons_score',
       'phyloP17way_primate_score', 'phastCons17way_primate_score',
       'bStatistic_score', 'n_methods_having_preds'],

In [4]:
def print_missing_things(x:pd.DataFrame):
    model_names = ['MetaRNN_score', 'MVP_score', 'SIFT_score', 'Polyphen2_HVAR_score', 'CADD_raw_score', 'REVEL_score', 
                'integrated_fitCons_score', 'phyloP17way_primate_score', 'phastCons17way_primate_score', 'bStatistic_score']

    # print("", "Common", "Rare", "Ultra-rare", "Singleton", sep="\t")
    print("\t", end="")
    for i, cls in enumerate(["Common", "Rare", "Ultra-rare", "Singleton"]):
        prots = x[(x["class"]==cls)]["prot_acc_version"].unique().shape[0]
        print(f"{cls}({prots})", end="\t")
    print()
    for model_name in model_names:
        print(model_name, end="\t")
        for i, cls in enumerate(["Common", "Rare", "Ultra-rare", "Singleton"]):
            missing = x[(x["class"]==cls) & pd.isna(x[model_name])].shape[0]
            not_missing = x[(x["class"]==cls) & ~pd.isna(x[model_name])].shape[0]
            total = x[(x["class"]==cls)].shape[0]
            
            print(f"{missing}/{not_missing}", end="\t")
            if i==3: print()

print_missing_things(result_df)

	Common(6988)	Rare(10451)	Ultra-rare(15772)	Singleton(15868)	
MetaRNN_score	0/14429	0/25554	0/286752	0/422813	
MVP_score	11473/2956	2090/23464	2728/284024	3274/419539	
SIFT_score	1355/13074	1834/23720	12014/274738	17224/405589	
Polyphen2_HVAR_score	2042/12387	2895/22659	17669/269083	25107/397706	
CADD_raw_score	0/14429	0/25554	0/286752	0/422813	
REVEL_score	1399/13030	1907/23647	10789/275963	15214/407599	
integrated_fitCons_score	647/13782	1079/24475	8585/278167	6677/416136	
phyloP17way_primate_score	8/14421	5/25549	18/286734	21/422792	
phastCons17way_primate_score	8/14421	5/25549	18/286734	21/422792	
bStatistic_score	408/14021	657/24897	4788/281964	6595/416218	


In [5]:
x = result_df.copy()
a = set(x[(x["class"]=="Common")]["prot_acc_version"].unique())
b = set(x[(x["class"]=="Rare")]["prot_acc_version"].unique())
c = set(x[(x["class"]=="Ultra-rare")]["prot_acc_version"].unique())
d = set(x[(x["class"]=="Singleton")]["prot_acc_version"].unique())

cmn_and_rare_prots = a.intersection(b)
cmn_and_ultrarare_prots = a.intersection(c)
cmn_and_singleton_prots = a.intersection(d)
cmn_prots = cmn_and_rare_prots.union(cmn_and_ultrarare_prots).union(cmn_and_singleton_prots)
cmn_rare_urare_singleton_prots = a.intersection(b).intersection(c).intersection(d)
print(len(cmn_and_rare_prots), len(cmn_and_ultrarare_prots), len(cmn_and_singleton_prots), len(cmn_prots), len(cmn_rare_urare_singleton_prots))

5521 6957 6955 6976 5503


In [11]:
def sample_variants(class_name, prots_to_cover):
    print(len(prots_to_cover))
    y = result_df[(result_df["class"]==class_name)].copy()
    rem = prots_to_cover

    for i in range(6, 1, -1):
        y_i = y[(y["prot_acc_version"].isin(rem)) & (y["n_methods_having_preds"]==i)] #rare-cmn=null
        y_i = y_i.groupby("prot_acc_version").sample(1)
        if i==6: data = y_i
        else: data = pd.concat([data, y_i])
        covered = set(data["prot_acc_version"].unique())
        rem = prots_to_cover - covered
        print("total covered: ", len(covered), "remained: ", len(rem))
        if len(rem)==0: break

    # print_missing_things(data)
    return data

# c = sample_variants("Common", cmn_rare_urare_singleton_prots)
# r = sample_variants("Rare", cmn_rare_urare_singleton_prots)
# ur = sample_variants("Ultra-rare", cmn_rare_urare_singleton_prots)
# s = sample_variants("Singleton", cmn_rare_urare_singleton_prots)

c = sample_variants("Common", cmn_prots)
r = sample_variants("Rare", cmn_and_rare_prots)
ur = sample_variants("Ultra-rare", cmn_and_ultrarare_prots)
s = sample_variants("Singleton", cmn_and_singleton_prots)

# test
# c_set = set(c["prot_acc_version"].unique())
# r_set = set(r["prot_acc_version"].unique())
# r_set - c_set # this should be empty

6976
total covered:  1440 remained:  5536
total covered:  6227 remained:  749
total covered:  6620 remained:  356
total covered:  6830 remained:  146
total covered:  6976 remained:  0
5521
total covered:  4724 remained:  797
total covered:  5176 remained:  345
total covered:  5330 remained:  191
total covered:  5461 remained:  60
total covered:  5521 remained:  0
6957
total covered:  6188 remained:  769
total covered:  6592 remained:  365
total covered:  6761 remained:  196
total covered:  6898 remained:  59
total covered:  6957 remained:  0
6955
total covered:  6197 remained:  758
total covered:  6594 remained:  361
total covered:  6764 remained:  191
total covered:  6894 remained:  61
total covered:  6955 remained:  0


In [12]:
good_data = pd.concat([c, r, ur, s])
print_missing_things(good_data)

c_set = set(good_data[good_data["class"]=="Common"]["prot_acc_version"].unique())
r_set = set(good_data[good_data["class"]=="Rare"]["prot_acc_version"].unique())
ur_set = set(good_data[good_data["class"]=="Ultra-rare"]["prot_acc_version"].unique())
s_set = set(good_data[good_data["class"]=="Singleton"]["prot_acc_version"].unique())
print(r_set - c_set)
print(ur_set - c_set)
print(s_set - c_set)

	Common(6976)	Rare(5521)	Ultra-rare(6957)	Singleton(6955)	
MetaRNN_score	0/6976	0/5521	0/6957	0/6955	
MVP_score	5263/1713	201/5320	104/6853	103/6852	
SIFT_score	449/6527	332/5189	349/6608	350/6605	
Polyphen2_HVAR_score	646/6330	520/5001	565/6392	555/6400	
CADD_raw_score	0/6976	0/5521	0/6957	0/6955	
REVEL_score	429/6547	340/5181	371/6586	363/6592	
integrated_fitCons_score	263/6713	185/5336	235/6722	222/6733	
phyloP17way_primate_score	1/6975	0/5521	0/6957	0/6955	
phastCons17way_primate_score	1/6975	0/5521	0/6957	0/6955	
bStatistic_score	144/6832	122/5399	137/6820	129/6826	
set()
set()
set()


In [13]:
result_df = good_data.copy()
print(result_df.columns)
print("total: ", result_df.shape[0])
print(result_df["class"].value_counts())

Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq', 'class', 'chrom', 'SIFT_score',
       'Polyphen2_HVAR_score', 'MetaRNN_score', 'REVEL_score', 'MVP_score',
       'CADD_raw_score', 'integrated_fitCons_score',
       'phyloP17way_primate_score', 'phastCons17way_primate_score',
       'bStatistic_score', 'n_methods_having_preds'],
      dtype='object')
total:  26409
Common        6976
Ultra-rare    6957
Singleton     6955
Rare          5521
Name: class, dtype: int64


In [14]:
result_df = good_data.copy()
out_filepath = home_dir+f"models/aa_common/datasets_popu_freq/popu_freq_with_dbnsfp_sampled"
result_df.to_csv(out_filepath+".tsv", sep="\t", index=False)
# result_df = pd.read_csv(out_filepath+".tsv", sep="\t")

# Creating merged fasta document ...
protein_acc_list = list(result_df["prot_acc_version"].unique())
print(len(protein_acc_list))
from utils.ncbi_proteins import create_combined_fasta
create_combined_fasta(protein_acc_list, out_filepath+".fasta", home_dir)

6976
0 NP_000015.2 Already existis
1 NP_000024.2 Already existis
2 NP_000053.2 Already existis
3 NP_000054.2 Already existis
4 NP_000068.1 Already existis
5 NP_000076.2 Already existis
6 NP_000086.2 Already existis
7 NP_000133.1 Already existis
8 NP_000144.2 Already existis
9 NP_000148.2 Already existis
10 NP_000156.1 Already existis
11 NP_000161.2 Already existis
12 NP_000162.2 Already existis
13 NP_000194.2 Already existis
14 NP_000209.2 Already existis
15 NP_000210.2 Already existis
16 NP_000249.1 Already existis
17 NP_000253.1 Already existis
18 NP_000256.4 Already existis
19 NP_000264.2 Already existis
20 NP_000265.1 Already existis
21 NP_000274.3 Already existis
22 NP_000280.1 Already existis
23 NP_000286.3 Already existis
24 NP_000292.1 Already existis
25 NP_000333.1 Already existis
26 NP_000337.1 Already existis
27 NP_000340.2 Already existis
28 NP_000352.1 Already existis
29 NP_000388.2 Already existis
30 NP_000408.1 Already existis
31 NP_000412.4 Already existis
32 NP_000422.

### the following needs to be deleted

In [ ]:
# temporarily added gene_symbol col
out_filepath = home_dir+f"models/aa_common/datasets_popu_freq/popu_freq_with_dbnsfp_sampled"
x = pd.read_csv(out_filepath+".tsv", sep="\t")
print(x.columns)
print(x["class"].value_counts())
print(x.shape)
cols = ['snp_id', 'gene_symbol', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele', 'prot_acc_version', 'prot_pos', 'wt', 'mut']
z = result_df[cols]
y = x.merge(z, how="inner", on=['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele', 'prot_acc_version', 'prot_pos', 'wt', 'mut'])
print(y.columns)
print(y["class"].value_counts())
print(y.shape)
print(y['gene_symbol'].unique().shape)

result_df.to_csv(out_filepath+"_.tsv", sep="\t", index=False)

In [34]:
y = result_df.copy()
y_6 = y[(y["prot_acc_version"].isin(cmn_and_rare_prots)) & (y["n_methods_having_preds"]==6)] #rare-cmn=null
remaining_cmn_prots = cmn_and_rare_prots - set(y_6[(y_6["class"]=="Common")]["prot_acc_version"].unique())
remaining_rare_prots = cmn_and_rare_prots - set(y_6[(y_6["class"]=="Rare")]["prot_acc_version"].unique())
print(len(remaining_cmn_prots), len(remaining_rare_prots)) # ? and ? common and rare prots do not have any variants yet, so take from 5
# remaining_prots = cmn_and_rare_prots - set(y_6["prot_acc_version"].unique()) # this is not enought, coz there can be prots for common but not for rare.
remaining_prots = remaining_cmn_prots.union(remaining_rare_prots)
print(len(remaining_prots))


# y_5 = y[(y["prot_acc_version"].isin(remaining_prots)) & (y["n_methods_having_preds"]==5)]
# y_6_5 = pd.concat([y_6, y_5])
# remaining_cmn_prots = cmn_and_rare_prots - set(y_6_5[(y_6_5["class"]=="Common")]["prot_acc_version"].unique())
# remaining_rare_prots = cmn_and_rare_prots - set(y_6_5[(y_6_5["class"]=="Rare")]["prot_acc_version"].unique())
# print(len(remaining_cmn_prots), len(remaining_rare_prots)) # ? and ? common and rare prots do not have any variants yet, so take from 4
# remaining_prots = remaining_cmn_prots.union(remaining_rare_prots)
# print(len(remaining_prots))

# y_4 = y[(y["prot_acc_version"].isin(remaining_prots)) & (y["n_methods_having_preds"]==4)]
# y_6_5_4 = pd.concat([y_6_5, y_4])
# remaining_cmn_prots = cmn_and_rare_prots - set(y_6_5_4[(y_6_5_4["class"]=="Common")]["prot_acc_version"].unique())
# remaining_rare_prots = cmn_and_rare_prots - set(y_6_5_4[(y_6_5_4["class"]=="Rare")]["prot_acc_version"].unique())
# print(len(remaining_cmn_prots), len(remaining_rare_prots)) # ? and ? common and rare prots do not have any variants yet, so take from 3
# remaining_prots = remaining_cmn_prots.union(remaining_rare_prots)
# print(len(remaining_prots))

# y_3 = y[(y["prot_acc_version"].isin(remaining_prots)) & (y["n_methods_having_preds"]==3)]
# y_6_5_4_3 = pd.concat([y_6_5_4, y_3])
# remaining_cmn_prots = cmn_and_rare_prots - set(y_6_5_4_3[(y_6_5_4_3["class"]=="Common")]["prot_acc_version"].unique())
# remaining_rare_prots = cmn_and_rare_prots - set(y_6_5_4_3[(y_6_5_4_3["class"]=="Rare")]["prot_acc_version"].unique())
# print(len(remaining_cmn_prots), len(remaining_rare_prots)) # ? and ? common and rare prots do not have any variants yet, so take from 3
# remaining_prots = remaining_cmn_prots.union(remaining_rare_prots)
# print(len(remaining_prots))
# print_missing_things(y_6_5_4_3)

# y_4 = y[(y["prot_acc_version"].isin(cmn_and_rare_prots)) & (y["n_methods_having_preds"]==4)]
# y_3 = y[(y["prot_acc_version"].isin(cmn_and_rare_prots)) & (y["n_methods_having_preds"]==3)]

4310 791
4337


In [33]:
c = set(y_6_5_4_3[(y_6_5_4_3["class"]=="Common")]["prot_acc_version"].unique())
r = set(y_6_5_4_3[(y_6_5_4_3["class"]=="Rare")]["prot_acc_version"].unique())
c-r

{'NP_001137355.1', 'NP_001164171.2', 'NP_001191406.1', 'NP_001334924.1'}

In [5]:
result_6_df = result_df[result_df["n_methods_having_preds"]==6]
result_5_df = result_df[result_df["n_methods_having_preds"]==5]
result_4_df = result_df[result_df["n_methods_having_preds"]==4]
result_3_df = result_df[result_df["n_methods_having_preds"]==3]
print(result_6_df.shape, result_5_df.shape, result_4_df.shape, result_3_df.shape)

(667123, 26) (47403, 26) (17366, 26) (9876, 26)


In [11]:
def print_data(result_x_df):
    print("#-of prots: ", result_x_df["prot_acc_version"].unique().shape[0])
    print("total: ", result_x_df.shape[0])
    print(result_x_df["class"].value_counts())
    print()

print_data(result_6_df)
print_data(result_5_df)
print_data(result_4_df)
print_data(result_3_df)

#-of prots:  14475
total:  667123
Singleton     384828
Ultra-rare    259706
Rare           20336
Common          2253
Name: class, dtype: int64

#-of prots:  7156
total:  47403
Singleton     20265
Ultra-rare    14266
Common         9836
Rare           3036
Name: class, dtype: int64

#-of prots:  991
total:  17366
Singleton     9092
Ultra-rare    6347
Common        1041
Rare           886
Name: class, dtype: int64

#-of prots:  571
total:  9876
Singleton     4922
Ultra-rare    3530
Rare           747
Common         677
Name: class, dtype: int64



In [77]:
result_3to5_df = result_df[(result_df["n_methods_having_preds"]>=3) & (result_df["n_methods_having_preds"]<=5)]
prots_6 = result_6_df["prot_acc_version"].unique()
mask = result_3to5_df["prot_acc_version"].isin(prots_6) & ((result_3to5_df["class"]=="Common") | (result_3to5_df["class"]=="Rare"))
result_3to5_commonrare_df = result_3to5_df[mask]
print(result_3to5_commonrare_df.shape)
print(result_3to5_commonrare_df["class"].value_counts())

result_6_expanded_df = pd.concat([result_6_df, result_3to5_commonrare_df])
print(result_6_expanded_df["class"].value_counts())
print(result_6_expanded_df["prot_acc_version"].unique().shape[0])

(11647, 26)
Common    9819
Rare      1828
Name: class, dtype: int64
Singleton     384828
Ultra-rare    259706
Rare           22164
Common         12072
Name: class, dtype: int64
14475


In [85]:
# recomputing the missing values to see how much reduced
x = result_6_expanded_df.copy()
model_names = ['MetaRNN_score', 'MVP_score', 'SIFT_score', 'Polyphen2_HVAR_score', 'CADD_raw_score', 'REVEL_score', 
               'integrated_fitCons_score', 'phyloP17way_primate_score', 'phastCons17way_primate_score', 'bStatistic_score']

# print("", "Common", "Rare", "Ultra-rare", "Singleton", sep="\t")
print("\t", end="")
for i, cls in enumerate(["Common", "Rare", "Ultra-rare", "Singleton"]):
    prots = x[(x["class"]==cls)]["prot_acc_version"].unique().shape[0]
    print(f"{cls}({prots})", end="\t")
print()
for model_name in model_names:
    print(model_name, end="\t")
    for i, cls in enumerate(["Common", "Rare", "Ultra-rare", "Singleton"]):
        missing = x[(x["class"]==cls) & pd.isna(x[model_name])].shape[0]
        not_missing = x[(x["class"]==cls) & ~pd.isna(x[model_name])].shape[0]
        total = x[(x["class"]==cls)].shape[0]
        
        print(f"{missing}/{not_missing}", end="\t")
        if i==3: print()

	Common(6164)	Rare(9404)	Ultra-rare(14331)	Singleton(14431)	
MetaRNN_score	0/12072	0/22164	0/259706	0/384828	
MVP_score	9736/2336	1385/20779	0/259706	0/384828	
SIFT_score	148/11924	251/21913	0/259706	0/384828	
Polyphen2_HVAR_score	173/11899	271/21893	0/259706	0/384828	
CADD_raw_score	0/12072	0/22164	0/259706	0/384828	
REVEL_score	70/12002	144/22020	0/259706	0/384828	
integrated_fitCons_score	279/11793	699/21465	6727/252979	4920/379908	
phyloP17way_primate_score	2/12070	3/22161	2/259704	2/384826	
phastCons17way_primate_score	2/12070	3/22161	2/259704	2/384826	
bStatistic_score	190/11882	398/21766	3724/255982	5351/379477	


In [4]:
# keeping only those that for which at leat 3 out of 6 supervised models' predictions.
result_df = result_df[result_df["n_methods_having_preds"]>=3]
print(result_df.columns)
print(result_df["class"].value_counts())
print(result_df.shape)

# saving for sampling neutral set for pathogenicity analysis.
result_df.to_csv(home_dir+f"models/aa_common/datasets_population_freq/popu_freq_with_dbnsfp.tsv", sep="\t", index=False)

Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq', 'class', 'chrom', 'SIFT_score',
       'Polyphen2_HVAR_score', 'MetaRNN_score', 'REVEL_score', 'MVP_score',
       'CADD_raw_score', 'integrated_fitCons_score',
       'phyloP17way_primate_score', 'phastCons17way_primate_score',
       'bStatistic_score', 'n_methods_having_preds'],
      dtype='object')
Singleton     419107
Ultra-rare    283849
Rare           25005
Common         13807
Name: class, dtype: int64
(741768, 26)


In [5]:
# recomputing the missing values to see how much reduced
model_names = ['MetaRNN_score', 'MVP_score', 'SIFT_score', 'Polyphen2_HVAR_score', 'CADD_raw_score', 'REVEL_score', 
               'integrated_fitCons_score', 'phyloP17way_primate_score', 'phastCons17way_primate_score', 'bStatistic_score']
for model_name in model_names:
    missing, total = result_df[pd.isna(result_df[model_name])].shape[0], result_df.shape[0]
    print(f"\t{model_name}: ({missing}/{total})*100 = {(missing / total) * 100:.4f}")

	MetaRNN_score: (0/741768)*100 = 0.0000
	MVP_score: (15404/741768)*100 = 2.0767
	SIFT_score: (28137/741768)*100 = 3.7932
	Polyphen2_HVAR_score: (43141/741768)*100 = 5.8160
	CADD_raw_score: (0/741768)*100 = 0.0000
	REVEL_score: (25081/741768)*100 = 3.3812
	integrated_fitCons_score: (15391/741768)*100 = 2.0749
	phyloP17way_primate_score: (40/741768)*100 = 0.0054
	phastCons17way_primate_score: (40/741768)*100 = 0.0054
	bStatistic_score: (12009/741768)*100 = 1.6190


In [6]:
# sampling variants for each protein haveing at least 1 variants for each class
from Bio import SeqIO

data = []
n_unique_prots = result_df['prot_acc_version'].unique()
for i, prot_acc_version in enumerate(n_unique_prots):
    x = result_df[result_df['prot_acc_version']==prot_acc_version].copy()
    class_dict = x["class"].value_counts().to_dict()
    if len(class_dict)==4:
        print(f"{i}|{n_unique_prots.shape[0]}", prot_acc_version, class_dict)
        filepath = home_dir+f"data/proteins/fastas/{prot_acc_version}.fasta" 
        seq_record = SeqIO.parse(filepath, format="fasta").__next__()
        seq = str(seq_record.seq)
        x["seq_len"] = len(seq)
        x["seq"] = seq
        data.append(x[x["class"]=="Common"].sample(1, random_state=29))
        data.append(x[x["class"]=="Rare"].sample(1, random_state=29))
        data.append(x[x["class"]=="Ultra-rare"].sample(1, random_state=29))
        data.append(x[x["class"]=="Singleton"].sample(1, random_state=29))
    # if i==10: break
    
data = pd.concat(data)
print(data.shape)
print(data.columns)
print("#-of proteins: ", data['prot_acc_version'].unique().shape[0])

1|15780 NP_064519.2 {'Singleton': 27, 'Ultra-rare': 18, 'Rare': 1, 'Common': 1}
3|15780 NP_004276.2 {'Ultra-rare': 105, 'Singleton': 77, 'Rare': 3, 'Common': 1}
4|15780 NP_758872.1 {'Singleton': 35, 'Ultra-rare': 28, 'Rare': 2, 'Common': 1}
6|15780 NP_065806.1 {'Singleton': 20, 'Ultra-rare': 18, 'Rare': 1, 'Common': 1}
9|15780 NP_001124389.2 {'Ultra-rare': 42, 'Singleton': 32, 'Rare': 21, 'Common': 10}
10|15780 NP_003357.2 {'Singleton': 29, 'Ultra-rare': 11, 'Rare': 2, 'Common': 1}
12|15780 NP_064560.2 {'Singleton': 56, 'Ultra-rare': 37, 'Rare': 2, 'Common': 2}
15|15780 NP_112228.1 {'Singleton': 13, 'Ultra-rare': 8, 'Rare': 4, 'Common': 3}
18|15780 NP_067022.1 {'Singleton': 41, 'Ultra-rare': 20, 'Rare': 2, 'Common': 1}
19|15780 NP_000693.1 {'Singleton': 37, 'Ultra-rare': 20, 'Common': 1, 'Rare': 1}
24|15780 NP_001171754.1 {'Singleton': 43, 'Ultra-rare': 34, 'Common': 5, 'Rare': 3}
27|15780 NP_079203.4 {'Singleton': 56, 'Ultra-rare': 26, 'Rare': 3, 'Common': 2}
29|15780 NP_001164225.1 {

In [19]:
result_df = data.copy()
out_filepath = home_dir+f"models/aa_common/datasets_population_freq/popu_freq_with_dbnsfp_sampled"
result_df.to_csv(out_filepath+".tsv", sep="\t", index=False)
# result_df = pd.read_csv(out_filepath+".tsv", sep="\t")

# Creating merged fasta document ...
protein_acc_list = list(result_df["prot_acc_version"].unique())
print(len(protein_acc_list))
from utils.ncbi_proteins import create_combined_fasta
create_combined_fasta(protein_acc_list, out_filepath+".fasta", home_dir)

5354
0 NP_064519.2 Already existis
1 NP_004276.2 Already existis
2 NP_758872.1 Already existis
3 NP_065806.1 Already existis
4 NP_001124389.2 Already existis
5 NP_003357.2 Already existis
6 NP_064560.2 Already existis
7 NP_112228.1 Already existis
8 NP_067022.1 Already existis
9 NP_000693.1 Already existis
10 NP_001171754.1 Already existis
11 NP_079203.4 Already existis
12 NP_001164225.1 Already existis
13 NP_079539.2 Already existis
14 NP_115827.1 Already existis
15 NP_001157912.1 Already existis
16 NP_002617.3 Already existis
17 NP_112599.2 Already existis
18 NP_001071174.1 Already existis
19 NP_001035940.1 Already existis
20 NP_073753.3 Already existis
21 NP_000180.2 Already existis
22 NP_788276.1 Already existis
23 NP_061732.1 Already existis
24 NP_001005356.1 Already existis
25 NP_060988.3 Already existis
26 NP_057118.1 Already existis
27 NP_001167594.1 Already existis
28 NP_001276880.1 Already existis
29 NP_004052.2 Already existis
30 NP_077740.1 Already existis
31 NP_001161840.1

In [20]:
# computing the missing values to see how much reduced
model_names = ['MetaRNN_score', 'MVP_score', 'SIFT_score', 'Polyphen2_HVAR_score', 'CADD_raw_score', 'REVEL_score', 
               'integrated_fitCons_score', 'phyloP17way_primate_score', 'phastCons17way_primate_score', 'bStatistic_score']
for model_name in model_names:
    missing, total = result_df[pd.isna(result_df[model_name])].shape[0], result_df.shape[0]
    print(f"\t{model_name}: ({missing}/{total})*100 = {(missing / total) * 100:.4f}")

print(result_df.columns)
print(result_df["class"].value_counts())
print(result_df.shape)

	MetaRNN_score: (0/21416)*100 = 0.0000
	MVP_score: (4692/21416)*100 = 21.9089
	SIFT_score: (975/21416)*100 = 4.5527
	Polyphen2_HVAR_score: (1686/21416)*100 = 7.8726
	CADD_raw_score: (0/21416)*100 = 0.0000
	REVEL_score: (952/21416)*100 = 4.4453
	integrated_fitCons_score: (626/21416)*100 = 2.9230
	phyloP17way_primate_score: (5/21416)*100 = 0.0233
	phastCons17way_primate_score: (5/21416)*100 = 0.0233
	bStatistic_score: (428/21416)*100 = 1.9985
Index(['snp_id', 'chrom_acc_version', 'chrom_pos', 'ref_allele', 'alt_allele',
       'prot_acc_version', 'prot_pos', 'wt', 'mut', 'wt_population',
       'mut_poulation', 'wt_freq', 'mt_freq', 'class', 'chrom', 'SIFT_score',
       'Polyphen2_HVAR_score', 'MetaRNN_score', 'REVEL_score', 'MVP_score',
       'CADD_raw_score', 'integrated_fitCons_score',
       'phyloP17way_primate_score', 'phastCons17way_primate_score',
       'bStatistic_score', 'n_methods_having_preds', 'seq_len', 'seq'],
      dtype='object')
Common        5354
Rare          5354
